<a href="https://colab.research.google.com/github/Trannguyenphucthinh222/README/blob/master/Bitcoin%20heatmap%20graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --upgrade google-cloud-bigquery
!pip install swifter

import pandas as pd
import numpy as np
import swifter
import requests
import sqlite3
import json
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
from itertools import product
from datetime import datetime,timedelta
from decimal import Decimal
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px
import xarray as xr
auth.authenticate_user()

In [ ]:

#2009-01-02
#2012-01-03
# get expected dataset
daily_supply = pd.io.gbq.read_gbq(f'''
 with big_table as
(select *
from `bigquery-public-data.crypto_bitcoin.transactions`
WHERE (block_timestamp_month >= date("2009-01-01") and block_timestamp_month <= date("2014-10-01"))
),

full_table as
(select *
from
      (select datetime,
          value,
          r as address
    from
          (select date(block_timestamp) as datetime,
                t.addresses as address,
                -cast(t.value as numeric)/1e8 as value
          from big_table
          left join unnest(inputs) t
          union all
          select date(block_timestamp),
                d.addresses,
                cast(d.value as numeric)/1e8
          from big_table
          left join unnest(outputs) d)
    left join unnest(address) r)
 where address not in (
    '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
    'bc1qgdjqv0av3q56jvd82tkdjpy7gdp9ut8tlqmgrpmv24sq90ecnvqqjwvw97',
    '39884E3j6KZj82FK4vcCrkUvWYL5MQaS3v',
    '3M219KR5vEneNb47ewrPfWyb5jQ2DjxRP6',
    '3LYJfcfHPXYJreMsASk2jkn69LWEYKzexb',
    '3LCGsSmfr24demGvriN4e3ft8wEcDuHFqh',
    'bc1qs5vdqkusz4v7qac8ynx0vt9jrekwuupx2fl5udp9jql3sr03z3gsr2mf0f',
    '3LQUu4v9z6KNch71j7kbj8GPeAGUo1FW6a',
    '3MgEAFWu1HKSnZ5ZsC8qf61ZW18xrP5pgd',
    '3FHNBLobJnbCTFTVakh5TXmEneyf5PT61B',
    '38UmuUqPCrFmQo4khkomQwZ4VbY2nZMJ67',
    'bc1qchctnvmdva5z9vrpxkkxck64v7nmzdtyxsrq64',
    '3JZq4atUahhuA9rLhXLMhhTo133J9rF97j',
    'bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h',
    '34HpHYiyQwg69gFmCq2BGHjF1DZnZnBeBP',
    '3FupZp77ySr7jwoLYEJ9mwzJpvoNBXsBnE',
    '3BMEXqGpG4FxBA1KWhRFufXfSTRgzfDBhJ',
    '1LnoZawVFFQihU8d8ntxLMpYheZUfyeVAK',
    '1JQULE6yHr9UaitLr4wahTwJN7DaMX7W1Z',
    '1DcT5Wij5tfb3oVViF8mA8p4WrG98ahZPT',
    '1CY7fykRLWXeSbKB885Kr4KjQxmDdvW923',
    '3EGdfMJbhPCnxN44SKNZ94AVt9wwULd67S',
    '39wVd42giU95ca39sEPkbPTpWygvsBDuA5',
    '12qTdZHx6f77aQ74CPCZGSY47VaRwYjVD8',
    '162bzZT2hJfv5Gm3ZmWfWfHJjCtMD6rHhw',
    '3H5JTt42K7RmZtromfTSefcMEFMMe18pMD',
    '36NkTqCAApfRJBKicQaqrdKs29g6hyE4LS',
    '1N52wHoVR79PMDishab2XmRHsbekCdGquK',
    'bc1q080rkmk3kj86pxvf5nkxecdrw6nrx3zzy9xl7q',
    '16rF2zwSJ9goQ9fZfYoti5LsUqqegb5RnA',
    '3J1oFuTTWhHGJF2vLorfvReLkirX1JtWJj',
    '3HroDXv8hmzKRtaSfBffRgedKpru8fgy6M',
    '3DwVjwVeJa9Z5Pu15WHNfKcDxY5tFUGfdx',
    'bc1q2qqqt87kh33s0er58akh7v9cwjgd83z5smh9rp',
    '3HcEUguUZ4vyyMAPWDPUDjLqz882jXwMfV',
    '3Kzh9qAqVWQhEsfQz7zEQL1EuSx5tyNLNS',
    'bc1quhruqrghgcca950rvhtrg7cpd7u8k6svpzgzmrjy8xyukacl5lkq0r8l2d',
    '3DVJfEsDTPkGDvqPCLC41X85L1B1DQWDyh',
    '3ANziRvoBdNGkmGopgyhvzPuBvcL8sRL7S'
 )
),

balance_table as (
select datetime,
       address,
       sum(sum(value)) over (partition by address order by datetime asc) as balance
from full_table
group by 1,2
),

missing_dates as(
select address,dates as datetime from
UNNEST(GENERATE_DATE_ARRAY('2009-01-07','2014-10-22', INTERVAL 1 DAY)) AS dates cross join (select distinct address from balance_table)
),

joined as (
select distinct missing_dates.address, missing_dates.datetime,balance
from  missing_dates
left join balance_table on missing_dates.address = balance_table.address
and  balance_table.datetime = missing_dates.datetime
),

continuos_balance as
(select * except(balance),
ifnull(last_value(balance ignore nulls) over(partition by address order by datetime),0) as balance_edited
from joined),


final_table as
(
select datetime,
       --segment,
       balance_edited,
       address,
       case
            when percentage_change > 1 then 1
            else percentage_change
      end as percentage_change_edited,
      balance_change

from
      (
      select datetime,
            address,
            -- case
            --       when balance_edited <= 1 then '1. =<1 BTC'
            --       when balance_edited > 1 and balance_edited <= 10 then '2. 1 - 10 BTC'
            --       when balance_edited > 10 and balance_edited <= 100 then '3. 10 - 100 BTC'
            --       when balance_edited > 100 and balance_edited <= 1000 then '4. 100 - 1000 BTC'
            --       when balance_edited > 1000 and balance_edited <= 10000 then '5. 1000 - 10,000 BTC'
            --       when balance_edited > 10000 and balance_edited <= 100000 then '6. 10,000 - 100,000 BTC'
            --       when balance_edited > 100000 then '7. >100,000 BTC'
            -- end as segment,
            balance_edited,
            balance_edited - (lag(balance_edited) over (partition by address order by datetime)) as balance_change,

            round(ifnull((balance_edited/if(ifnull(lag(balance_edited) over (partition by address order by datetime),0) = 0,1,ifnull(lag(balance_edited) over (partition by address order by datetime),0)) - 1), 0), 2)  as percentage_change
      from continuos_balance
      --where balance_edited > 0
)),

bitcoin_supply as (
select datetime,
      sum(sum(btc_minted)) over (order by datetime asc) as cumsum_btc
from
      (select datetime,
             block_cnt*block_reward as btc_minted
      from
      (select  date(block_timestamp) as datetime,
                  case
                        when date(block_timestamp) >= '2009-01-03' and date(block_timestamp) <= '2012-11-28' then 50
                        when date(block_timestamp) > '2012-11-28' and date(block_timestamp) <= '2016-07-09' then 25
                        when date(block_timestamp) > '2016-07-09' and date(block_timestamp) <= '2020-05-11' then 12.5
                        when date(block_timestamp) > '2020-05-11' and date(block_timestamp) <= current_date() then 6.25
                  end as block_reward,
                  count(distinct block_hash) as block_cnt
      FROM `bigquery-public-data.crypto_bitcoin.transactions`
      WHERE block_timestamp >= "2009-01-07" and block_timestamp <= "2014-10-22"
      group by 1,2))
group by 1
),

pre_final_table as
(
select datetime,
         balance_edited,
         cumsum_btc,
         balance_change,
         lag(percentage_change_final,1) over (partition by address order by datetime desc) as percentage_change_last
  from
    (  select x.datetime,
          x.address,
          x.balance_edited,
          y.cumsum_btc,
          x.balance_change,
          case
                when x.balance_edited = 0 and x.balance_change = 0 then 0
                else x.percentage_change_edited
          end as percentage_change_final
    from final_table x left join bitcoin_supply y on x.datetime = y.datetime)

),

table_final as
(select datetime,
        sum((balance_edited/cumsum_btc)* percentage_change_last) as participant_ratio_point
    from pre_final_table
    group by 1),

acc_score_table as
    (select a.datetime,
          sum(a.participant_ratio_point) over (order by a.datetime) as accum_score
    from table_final a, table_final b
    where b.datetime < a.datetime AND b.datetime >= a.datetime - 30
    )

select distinct datetime,
       (accum_score - min_record)/(case when (max_record - min_record) = 0 then 1 else (max_record - min_record) end) as normalized_value
from
  (
      select distinct datetime,
        accum_score,
        max(accum_score) over (partition by date_trunc(datetime,quarter)) as max_record,
        min(accum_score) over (partition by date_trunc(datetime,quarter)) as min_record
  from acc_score_table
)
  ''',
  project_id = 'internal-blockchain-indexed'
)

In [ ]:
#without normalized


#2009-01-02
#2012-01-03
# get expected dataset
daily_supply_without_normalize = pd.io.gbq.read_gbq(f'''
 with big_table as
(select *
from `bigquery-public-data.crypto_bitcoin.transactions`
WHERE (block_timestamp_month >= date("2009-01-01") and block_timestamp_month <= date("2011-10-01"))
),

full_table as
(select *
from
      (select datetime,
          value,
          r as address
    from
          (select date(block_timestamp) as datetime,
                t.addresses as address,
                -cast(t.value as numeric)/1e8 as value
          from big_table
          left join unnest(inputs) t
          union all
          select date(block_timestamp),
                d.addresses,
                cast(d.value as numeric)/1e8
          from big_table
          left join unnest(outputs) d)
    left join unnest(address) r)
 where address not in (
    '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
    'bc1qgdjqv0av3q56jvd82tkdjpy7gdp9ut8tlqmgrpmv24sq90ecnvqqjwvw97',
    '39884E3j6KZj82FK4vcCrkUvWYL5MQaS3v',
    '3M219KR5vEneNb47ewrPfWyb5jQ2DjxRP6',
    '3LYJfcfHPXYJreMsASk2jkn69LWEYKzexb',
    '3LCGsSmfr24demGvriN4e3ft8wEcDuHFqh',
    'bc1qs5vdqkusz4v7qac8ynx0vt9jrekwuupx2fl5udp9jql3sr03z3gsr2mf0f',
    '3LQUu4v9z6KNch71j7kbj8GPeAGUo1FW6a',
    '3MgEAFWu1HKSnZ5ZsC8qf61ZW18xrP5pgd',
    '3FHNBLobJnbCTFTVakh5TXmEneyf5PT61B',
    '38UmuUqPCrFmQo4khkomQwZ4VbY2nZMJ67',
    'bc1qchctnvmdva5z9vrpxkkxck64v7nmzdtyxsrq64',
    '3JZq4atUahhuA9rLhXLMhhTo133J9rF97j',
    'bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h',
    '34HpHYiyQwg69gFmCq2BGHjF1DZnZnBeBP',
    '3FupZp77ySr7jwoLYEJ9mwzJpvoNBXsBnE',
    '3BMEXqGpG4FxBA1KWhRFufXfSTRgzfDBhJ',
    '1LnoZawVFFQihU8d8ntxLMpYheZUfyeVAK',
    '1JQULE6yHr9UaitLr4wahTwJN7DaMX7W1Z',
    '1DcT5Wij5tfb3oVViF8mA8p4WrG98ahZPT',
    '1CY7fykRLWXeSbKB885Kr4KjQxmDdvW923',
    '3EGdfMJbhPCnxN44SKNZ94AVt9wwULd67S',
    '39wVd42giU95ca39sEPkbPTpWygvsBDuA5',
    '12qTdZHx6f77aQ74CPCZGSY47VaRwYjVD8',
    '162bzZT2hJfv5Gm3ZmWfWfHJjCtMD6rHhw',
    '3H5JTt42K7RmZtromfTSefcMEFMMe18pMD',
    '36NkTqCAApfRJBKicQaqrdKs29g6hyE4LS',
    '1N52wHoVR79PMDishab2XmRHsbekCdGquK',
    'bc1q080rkmk3kj86pxvf5nkxecdrw6nrx3zzy9xl7q',
    '16rF2zwSJ9goQ9fZfYoti5LsUqqegb5RnA',
    '3J1oFuTTWhHGJF2vLorfvReLkirX1JtWJj',
    '3HroDXv8hmzKRtaSfBffRgedKpru8fgy6M',
    '3DwVjwVeJa9Z5Pu15WHNfKcDxY5tFUGfdx',
    'bc1q2qqqt87kh33s0er58akh7v9cwjgd83z5smh9rp',
    '3HcEUguUZ4vyyMAPWDPUDjLqz882jXwMfV',
    '3Kzh9qAqVWQhEsfQz7zEQL1EuSx5tyNLNS',
    'bc1quhruqrghgcca950rvhtrg7cpd7u8k6svpzgzmrjy8xyukacl5lkq0r8l2d',
    '3DVJfEsDTPkGDvqPCLC41X85L1B1DQWDyh',
    '3ANziRvoBdNGkmGopgyhvzPuBvcL8sRL7S'
 )
),

balance_table as (
select datetime,
       address,
       sum(sum(value)) over (partition by address order by datetime asc) as balance
from full_table
group by 1,2
),

missing_dates as(
select address,dates as datetime from
UNNEST(GENERATE_DATE_ARRAY('2009-01-07','2011-10-22', INTERVAL 1 DAY)) AS dates cross join (select distinct address from balance_table)
),

joined as (
select distinct missing_dates.address, missing_dates.datetime,balance
from  missing_dates
left join balance_table on missing_dates.address = balance_table.address
and  balance_table.datetime = missing_dates.datetime
),

continuos_balance as
(select * except(balance),
ifnull(last_value(balance ignore nulls) over(partition by address order by datetime),0) as balance_edited
from joined),


final_table as
(
select datetime,
       --segment,
       balance_edited,
       address,
       case
            when percentage_change > 1 then 1
            else percentage_change
      end as percentage_change_edited,
      balance_change

from
      (
      select datetime,
            address,
            -- case
            --       when balance_edited <= 1 then '1. =<1 BTC'
            --       when balance_edited > 1 and balance_edited <= 10 then '2. 1 - 10 BTC'
            --       when balance_edited > 10 and balance_edited <= 100 then '3. 10 - 100 BTC'
            --       when balance_edited > 100 and balance_edited <= 1000 then '4. 100 - 1000 BTC'
            --       when balance_edited > 1000 and balance_edited <= 10000 then '5. 1000 - 10,000 BTC'
            --       when balance_edited > 10000 and balance_edited <= 100000 then '6. 10,000 - 100,000 BTC'
            --       when balance_edited > 100000 then '7. >100,000 BTC'
            -- end as segment,
            balance_edited,
            balance_edited - (lag(balance_edited) over (partition by address order by datetime)) as balance_change,

            round(ifnull((balance_edited/if(ifnull(lag(balance_edited) over (partition by address order by datetime),0) = 0,1,ifnull(lag(balance_edited) over (partition by address order by datetime),0)) - 1), 0), 2)  as percentage_change
      from continuos_balance
      --where balance_edited > 0
)),

bitcoin_supply as (
select datetime,
      sum(sum(btc_minted)) over (order by datetime asc) as cumsum_btc
from
      (select datetime,
             block_cnt*block_reward as btc_minted
      from
      (select  date(block_timestamp) as datetime,
                  case
                        when date(block_timestamp) >= '2009-01-03' and date(block_timestamp) <= '2012-11-28' then 50
                        when date(block_timestamp) > '2012-11-28' and date(block_timestamp) <= '2016-07-09' then 25
                        when date(block_timestamp) > '2016-07-09' and date(block_timestamp) <= '2020-05-11' then 12.5
                        when date(block_timestamp) > '2020-05-11' and date(block_timestamp) <= current_date() then 6.25
                  end as block_reward,
                  count(distinct block_hash) as block_cnt
      FROM `bigquery-public-data.crypto_bitcoin.transactions`
      WHERE block_timestamp >= "2009-01-07" and block_timestamp <= "2011-10-22"
      group by 1,2))
group by 1
),

pre_final_table as
# (
# select datetime,
#          balance_edited,
#          cumsum_btc,
#          balance_change,
#          lag(percentage_change_final,1) over (partition by address order by datetime desc) as percentage_change_last
#   from
#     (
     (select x.datetime,
          x.address,
          x.balance_edited,
          y.cumsum_btc,
          x.balance_change,
          case
                when x.balance_edited = 0 and x.balance_change = 0 then 0
                else x.percentage_change_edited
          end as percentage_change_final
    from final_table x left join bitcoin_supply y on x.datetime = y.datetime)

--)
,

table_final as
(select datetime,
        sum((balance_edited/cumsum_btc)* percentage_change_final) as participant_ratio_point
    from pre_final_table
    group by 1)

--,

# acc_score_table as
#     (
    select a.datetime,
          sum(a.participant_ratio_point) as accum_score
    from table_final a, table_final b
    where b.datetime < a.datetime AND b.datetime >= a.datetime - 30
    group by 1
   -- )

# select distinct datetime,
#        (accum_score - min_record)/(case when (max_record - min_record) = 0 then 1 else (max_record - min_record) end) as normalized_value
# from
#   (
#       select distinct datetime,
#         accum_score,
#         max(accum_score) over (partition by date_trunc(datetime,quarter)) as max_record,
#         min(accum_score) over (partition by date_trunc(datetime,quarter)) as min_record
#   from acc_score_table
# )
  ''',
  project_id = 'internal-blockchain-indexed'
)

In [ ]:
#without normalized, daily


#2009-01-02
#2012-01-03
# get expected dataset
daily_supply_without_normalize_daily = pd.io.gbq.read_gbq(f'''
 with big_table as
(select *
from `bigquery-public-data.crypto_bitcoin.transactions`
WHERE (block_timestamp_month >= date("2009-01-01") and block_timestamp_month <= date("2011-10-01"))
),

full_table as
(select *
from
      (select datetime,
          value,
          r as address
    from
          (select date(block_timestamp) as datetime,
                t.addresses as address,
                -cast(t.value as numeric)/1e8 as value
          from big_table
          left join unnest(inputs) t
          union all
          select date(block_timestamp),
                d.addresses,
                cast(d.value as numeric)/1e8
          from big_table
          left join unnest(outputs) d)
    left join unnest(address) r)
 where address not in (
    '34xp4vRoCGJym3xR7yCVPFHoCNxv4Twseo',
    'bc1qgdjqv0av3q56jvd82tkdjpy7gdp9ut8tlqmgrpmv24sq90ecnvqqjwvw97',
    '39884E3j6KZj82FK4vcCrkUvWYL5MQaS3v',
    '3M219KR5vEneNb47ewrPfWyb5jQ2DjxRP6',
    '3LYJfcfHPXYJreMsASk2jkn69LWEYKzexb',
    '3LCGsSmfr24demGvriN4e3ft8wEcDuHFqh',
    'bc1qs5vdqkusz4v7qac8ynx0vt9jrekwuupx2fl5udp9jql3sr03z3gsr2mf0f',
    '3LQUu4v9z6KNch71j7kbj8GPeAGUo1FW6a',
    '3MgEAFWu1HKSnZ5ZsC8qf61ZW18xrP5pgd',
    '3FHNBLobJnbCTFTVakh5TXmEneyf5PT61B',
    '38UmuUqPCrFmQo4khkomQwZ4VbY2nZMJ67',
    'bc1qchctnvmdva5z9vrpxkkxck64v7nmzdtyxsrq64',
    '3JZq4atUahhuA9rLhXLMhhTo133J9rF97j',
    'bc1qm34lsc65zpw79lxes69zkqmk6ee3ewf0j77s3h',
    '34HpHYiyQwg69gFmCq2BGHjF1DZnZnBeBP',
    '3FupZp77ySr7jwoLYEJ9mwzJpvoNBXsBnE',
    '3BMEXqGpG4FxBA1KWhRFufXfSTRgzfDBhJ',
    '1LnoZawVFFQihU8d8ntxLMpYheZUfyeVAK',
    '1JQULE6yHr9UaitLr4wahTwJN7DaMX7W1Z',
    '1DcT5Wij5tfb3oVViF8mA8p4WrG98ahZPT',
    '1CY7fykRLWXeSbKB885Kr4KjQxmDdvW923',
    '3EGdfMJbhPCnxN44SKNZ94AVt9wwULd67S',
    '39wVd42giU95ca39sEPkbPTpWygvsBDuA5',
    '12qTdZHx6f77aQ74CPCZGSY47VaRwYjVD8',
    '162bzZT2hJfv5Gm3ZmWfWfHJjCtMD6rHhw',
    '3H5JTt42K7RmZtromfTSefcMEFMMe18pMD',
    '36NkTqCAApfRJBKicQaqrdKs29g6hyE4LS',
    '1N52wHoVR79PMDishab2XmRHsbekCdGquK',
    'bc1q080rkmk3kj86pxvf5nkxecdrw6nrx3zzy9xl7q',
    '16rF2zwSJ9goQ9fZfYoti5LsUqqegb5RnA',
    '3J1oFuTTWhHGJF2vLorfvReLkirX1JtWJj',
    '3HroDXv8hmzKRtaSfBffRgedKpru8fgy6M',
    '3DwVjwVeJa9Z5Pu15WHNfKcDxY5tFUGfdx',
    'bc1q2qqqt87kh33s0er58akh7v9cwjgd83z5smh9rp',
    '3HcEUguUZ4vyyMAPWDPUDjLqz882jXwMfV',
    '3Kzh9qAqVWQhEsfQz7zEQL1EuSx5tyNLNS',
    'bc1quhruqrghgcca950rvhtrg7cpd7u8k6svpzgzmrjy8xyukacl5lkq0r8l2d',
    '3DVJfEsDTPkGDvqPCLC41X85L1B1DQWDyh',
    '3ANziRvoBdNGkmGopgyhvzPuBvcL8sRL7S'
 )
),

balance_table as (
select datetime,
       address,
       sum(sum(value)) over (partition by address order by datetime asc) as balance
from full_table
group by 1,2
),

missing_dates as(
select address,dates as datetime from
UNNEST(GENERATE_DATE_ARRAY('2009-01-07','2011-10-22', INTERVAL 1 DAY)) AS dates cross join (select distinct address from balance_table)
),

joined as (
select distinct missing_dates.address, missing_dates.datetime,balance
from  missing_dates
left join balance_table on missing_dates.address = balance_table.address
and  balance_table.datetime = missing_dates.datetime
),

continuos_balance as
(select * except(balance),
ifnull(last_value(balance ignore nulls) over(partition by address order by datetime),0) as balance_edited
from joined),


final_table as
(
select datetime,
       --segment,
       balance_edited,
       address,
       case
            when percentage_change > 1 then 1
            else percentage_change
      end as percentage_change_edited,
      balance_change

from
      (
      select datetime,
            address,
            -- case
            --       when balance_edited <= 1 then '1. =<1 BTC'
            --       when balance_edited > 1 and balance_edited <= 10 then '2. 1 - 10 BTC'
            --       when balance_edited > 10 and balance_edited <= 100 then '3. 10 - 100 BTC'
            --       when balance_edited > 100 and balance_edited <= 1000 then '4. 100 - 1000 BTC'
            --       when balance_edited > 1000 and balance_edited <= 10000 then '5. 1000 - 10,000 BTC'
            --       when balance_edited > 10000 and balance_edited <= 100000 then '6. 10,000 - 100,000 BTC'
            --       when balance_edited > 100000 then '7. >100,000 BTC'
            -- end as segment,
            balance_edited,
            balance_edited - (lag(balance_edited) over (partition by address order by datetime)) as balance_change,

            round(ifnull((balance_edited/if(ifnull(lag(balance_edited) over (partition by address order by datetime),0) = 0,1,ifnull(lag(balance_edited) over (partition by address order by datetime),0)) - 1), 0), 2)  as percentage_change
      from continuos_balance
      --where balance_edited > 0
)),

bitcoin_supply as (
select datetime,
      sum(sum(btc_minted)) over (order by datetime asc) as cumsum_btc
from
      (select datetime,
             block_cnt*block_reward as btc_minted
      from
      (select  date(block_timestamp) as datetime,
                  case
                        when date(block_timestamp) >= '2009-01-03' and date(block_timestamp) <= '2012-11-28' then 50
                        when date(block_timestamp) > '2012-11-28' and date(block_timestamp) <= '2016-07-09' then 25
                        when date(block_timestamp) > '2016-07-09' and date(block_timestamp) <= '2020-05-11' then 12.5
                        when date(block_timestamp) > '2020-05-11' and date(block_timestamp) <= current_date() then 6.25
                  end as block_reward,
                  count(distinct block_hash) as block_cnt
      FROM `bigquery-public-data.crypto_bitcoin.transactions`
      WHERE block_timestamp >= "2009-01-07" and block_timestamp <= "2014-10-22"
      group by 1,2))
group by 1
),

pre_final_table as
(
# select datetime,
#          balance_edited,
#          cumsum_btc,
#          balance_change,
#          lag(percentage_change_final,1) over (partition by address order by datetime desc) as percentage_change_last
#   from
#     (
      select x.datetime,
          x.address,
          x.balance_edited,
          y.cumsum_btc,
          x.balance_change,
          case
                when x.balance_edited = 0 and x.balance_change = 0 then 0
                else x.percentage_change_edited
          end as percentage_change_final
    from final_table x left join bitcoin_supply y on x.datetime = y.datetime)

--)
--,

# table_final as
# (
  select datetime,
        sum((balance_edited/cumsum_btc)* percentage_change_final) as participant_ratio_point
    from pre_final_table
    group by 1
    --)

--,

# acc_score_table as
#     (
    # select a.datetime,
    #       sum(a.participant_ratio_point) over (order by a.datetime) as accum_score
    # from table_final a, table_final b
    # where b.datetime < a.datetime AND b.datetime >= a.datetime - 30

   -- )

# select distinct datetime,
#        (accum_score - min_record)/(case when (max_record - min_record) = 0 then 1 else (max_record - min_record) end) as normalized_value
# from
#   (
#       select distinct datetime,
#         accum_score,
#         max(accum_score) over (partition by date_trunc(datetime,quarter)) as max_record,
#         min(accum_score) over (partition by date_trunc(datetime,quarter)) as min_record
#   from acc_score_table
# )
  ''',
  project_id = 'internal-blockchain-indexed'
)

In [ ]:
fulltime_btc_heatmap = pd.io.gbq.read_gbq(f'''
 select *
from `internal-blockchain-indexed.analytics_data.btc_heatmap_data`
union all
select *
from `internal-blockchain-indexed.analytics_data.btc_heatmap_data_t10`
  ''',
  project_id = 'internal-blockchain-indexed'
)

In [ ]:
table_copy = daily_supply.copy()

In [ ]:
data_path_2 ="/content/btc_price_all_time.csv"  #"/content/bitcoin_price_2.csv"

In [ ]:
price = pd.read_csv(data_path_2, usecols = ['End','Close'])

In [ ]:
price_copy = price.copy()
price_final = price_copy.rename(columns={"End": "datetime"}, errors="raise")

In [ ]:
table_copy['datetime'] = pd.to_datetime(table_copy['datetime'])
price_final['datetime'] = pd.to_datetime(price_final['datetime'])
price_edited = price_copy.sort_values('End', ascending=True)
table_copy['normalized_value']=table_copy['normalized_value'].astype(float)
result = pd.merge(price_final,table_copy ,how="right", on= "datetime")
result_edited = result.fillna(0)

In [ ]:
fig = px.scatter(result_edited, x="datetime", y="Close", color='normalized_value',color_continuous_scale='inferno_r',log_y=True)
fig.show()

In [ ]:
table_copy_2 = daily_supply_without_normalize.copy()

In [ ]:
table_copy_2['datetime'] = pd.to_datetime(table_copy_2['datetime'])
price_final['datetime'] = pd.to_datetime(price_final['datetime'])
price_edited = price_copy.sort_values('End', ascending=True)
table_copy_2['accum_score']=table_copy_2['accum_score'].astype(float)
result_2 = pd.merge(price_final,table_copy_2 ,how="right", on= "datetime")
result_edited_2 = result_2.fillna(0)

In [ ]:
fig_2 = px.scatter(result_edited_2, x="datetime", y="Close", color='accum_score',color_continuous_scale='inferno_r',log_y=True,range_color=[0,1])
fig_2.show()

In [ ]:
table_copy_3 = daily_supply_without_normalize_daily.copy()


In [ ]:
table_copy_3['datetime'] = pd.to_datetime(table_copy_3['datetime'])
price_final['datetime'] = pd.to_datetime(price_final['datetime'])
price_edited = price_copy.sort_values('End', ascending=True)
table_copy_3['participant_ratio_point']=table_copy_3['participant_ratio_point'].astype(float)
result_3 = pd.merge(price_final,table_copy_3 ,how="right", on= "datetime")
result_edited_3 = result_3.fillna(0)

In [ ]:
fig_3 = px.scatter(result_edited_3, x="datetime", y="Close", color='participant_ratio_point',color_continuous_scale='inferno_r',log_y=True,range_color=[0.003,0.1])
fig_3.show()

In [ ]:
table_copy_4 = fulltime_btc_heatmap.copy()

In [ ]:
table_copy_4['datetime'] = pd.to_datetime(table_copy_4['datetime'])
price_final['datetime'] = pd.to_datetime(price_final['datetime'])
price_edited = price_copy.sort_values('End', ascending=True)
table_copy_4['normalized_value']=table_copy_4['normalized_value'].astype(float)
result_4 = pd.merge(price_final,table_copy_4 ,how="right", on= "datetime")
result_edited_4 = result_4.fillna(0)

In [ ]:
fig_4 = px.scatter(result_edited_4, x="datetime", y="Close", color='normalized_value',color_continuous_scale='inferno_r',log_y=True,range_color=[0,1])
fig_4.show()